In [1]:
import os
import pandas as pd
import numpy as np
from configs.configuration import Config
from sklearn.metrics.pairwise import cosine_similarity
import cv2

from utils.get_data import get_npy
from utils.top_k_accuracy import accuracy

np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

from glob import glob
import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
import matplotlib.pyplot as plt

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
from model.resnet18_lambda1 import ResNet18_lambda

In [4]:
config = Config(os.path.join(os.getcwd(), "configs/resnet18-mcpl-ranlars.yaml"))
Embedding = pd.read_csv(config.embed.path, index_col=0)

---------------------------------- APP CONFIG ----------------------------------
model: 
  name: resnet18-lambda-mcpl-ranlars-aug-embed2_4808
  pretrain: False
zero_shot: False
embed: 
  path: data/embed/Embedding_tc_0.3_0.001_0.6.csv
loss: 
  name: MCPL
train: 
  source: data/cha_table/tc_cha_4808.txt
  optim: ralars
  lr: 0.005
  max_epoch: 100
  batch_size: 512
  data_shuffle: True
test: 
  source: cha_table/cha_test-Copy1.txt
  batch_size: 512
  data_shuffle: False
aug: 
  aug_func: False
  train_amplify: 5
  test_amplify: 1
--------------------------------------------------------------------------------


In [5]:
def infer(imgs):
    
    imgs = imgs/255
    imgs = torch.tensor(imgs, dtype=torch.float)
    imgs = Variable(imgs.cuda())
    imgs = torch.unsqueeze(imgs, 1)
    
    pred_y = model(imgs)
    return pred_y.cpu().detach().numpy()

In [6]:
model = ResNet18_lambda(out_dim=301).cuda()

### reload weights
checkpoint = torch.load('save/model_3755cha_allembed_resnet18+lambda_MCPL_ralars_aug_4808_embed2.h5')
model.load_state_dict(checkpoint['model_state_dict'])

with torch.no_grad():
    model.eval()

RuntimeError: Error(s) in loading state_dict for ResNet18_lambda:
	Missing key(s) in state_dict: "lalayer.rel_pos_emb". 
	Unexpected key(s) in state_dict: "lalayer.pos_emb". 
	size mismatch for lalayer.to_q.weight: copying a param with shape torch.Size([64, 1000, 1, 1]) from checkpoint, the shape in current model is torch.Size([64, 256, 1, 1]).
	size mismatch for lalayer.to_k.weight: copying a param with shape torch.Size([64, 1000, 1, 1]) from checkpoint, the shape in current model is torch.Size([64, 256, 1, 1]).
	size mismatch for lalayer.to_v.weight: copying a param with shape torch.Size([1000, 1000, 1, 1]) from checkpoint, the shape in current model is torch.Size([256, 256, 1, 1]).
	size mismatch for lalayer.norm_v.weight: copying a param with shape torch.Size([1000]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for lalayer.norm_v.bias: copying a param with shape torch.Size([1000]) from checkpoint, the shape in current model is torch.Size([256]).


In [ ]:
img = cv2.imread('data/crop3/tsmc-sample-invoices_Invoice_JPEG_Gray_HW_TC_Invoice_ (5).jpg', cv2.IMREAD_GRAYSCALE)
plt.figure(figsize=(20,20))
plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
crops_path = glob('data/crop3/*.png')

imgs = []
raw_imgs = []
kernel = np.ones((2,2),np.uint8)

for path in crops_path:
    crop = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    raw_imgs.append(crop)
    
    # erosion & dilation
    # erosion = cv2.erode(crop, kernel, iterations = 1)
    # dilation = cv2.dilate(erosion, kernel, iterations = 1)
    # erosion = cv2.medianBlur(crop, 3)

    resImg = cv2.resize(crop, (64,64), interpolation=cv2.INTER_CUBIC)
    imgs.append(resImg)
    


pred_y = infer(np.array(imgs))

pred_y = np.vstack(pred_y)
score = cosine_similarity(pred_y, Embedding.values.T)
top_score = score.argsort()[:,::-1][:,:10]

for idx, n in enumerate(pred_y):
    print(Embedding.columns[top_score][idx])
#     plt.imshow(raw_imgs[idx], cmap='gray')
#     plt.show()
    
    plt.imshow(imgs[idx], cmap='gray')
    plt.show()

In [ ]:
crops_path = glob('data/crop2/*.png')

imgs = []
raw_imgs = []
kernel = np.ones((2,2),np.uint8)

for path in crops_path:
    crop = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    raw_imgs.append(crop)
    
    # erosion & dilation
    # erosion = cv2.erode(crop, kernel, iterations = 1)
    # dilation = cv2.dilate(crop, kernel, iterations = 1)
    # erosion = cv2.medianBlur(crop, 3)

    resImg = cv2.resize(crop, (64,64), interpolation=cv2.INTER_CUBIC)
    imgs.append(resImg)
    


pred_y = infer(np.array(imgs))

pred_y = np.vstack(pred_y)
score = cosine_similarity(pred_y, Embedding.values.T)
top_score = score.argsort()[:,::-1][:,:10]

for idx, n in enumerate(pred_y):
    print(Embedding.columns[top_score][idx])
#     plt.imshow(raw_imgs[idx], cmap='gray')
#     plt.show()
    
    plt.imshow(imgs[idx], cmap='gray')
    plt.show()

In [ ]:
crops_path = glob('data/crop/*.png')

imgs = []
raw_imgs = []
kernel = np.ones((2,2),np.uint8)

for path in crops_path:
    crop = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    raw_imgs.append(crop)
    
    # erosion & dilation
    # erosion = cv2.erode(crop, kernel, iterations = 1)
    # dilation = cv2.dilate(crop, kernel, iterations = 1)
    # erosion = cv2.medianBlur(crop, 3)

    resImg = cv2.resize(crop, (64,64), interpolation=cv2.INTER_CUBIC)
    imgs.append(resImg)
    


pred_y = infer(np.array(imgs))

pred_y = np.vstack(pred_y)
score = cosine_similarity(pred_y, Embedding.values.T)
top_score = score.argsort()[:,::-1][:,:10]

for idx, n in enumerate(pred_y):
    print(Embedding.columns[top_score][idx])
#     plt.imshow(raw_imgs[idx], cmap='gray')
#     plt.show()
    
    plt.imshow(imgs[idx], cmap='gray')
    plt.show()